In [155]:
import json
import requests
import re
import pandas as pd

In [156]:
response = requests.get("https://twitter-dashboard-sentiment-default-rtdb.firebaseio.com/Pets.json")
response

<Response [200]>

In [157]:
data = json.loads(response.content.decode(response.encoding))

In [158]:
def debug(arg):
    print('type: ',type(arg))
    print('len: ',len(arg))
    print(arg)

In [159]:
def extract(data_arg,what_arg,from_arg):
    info=[]
    for d in data_arg.values():
        info.append([d[from_arg], d[what_arg]])
    df=pd.DataFrame(info,columns=[from_arg,what_arg])
    return df

1.	El día de la semana en el que más se habló de una mascota.

In [160]:
df_day=extract(data,'day','pet')
debug(df_day)

type:  <class 'pandas.core.frame.DataFrame'>
len:  8012
          pet      day
0     parrots   Monday
1     parrots   Monday
2     parrots   Monday
3     parrots   Monday
4     parrots   Monday
...       ...      ...
8007     cats  Tuesday
8008     cats  Tuesday
8009     cats  Tuesday
8010     cats  Tuesday
8011     cats  Tuesday

[8012 rows x 2 columns]


In [161]:
def populate_model(arg_df,w_arg,f_arg):
    # TODO:parametrizar argumentos de pet y dia para reusar
    for pet in arg_df[f_arg].unique():
        counter=0
        df=arg_df[arg_df.pet  == pet]
        activity = df.pivot_table(index=[w_arg], aggfunc='size')
        for day in df[w_arg].unique():
            count=activity[day]
            if count>counter:
                counter=count
                print(pet,day,count)
                # model_day= day(pet=pet, day=day, count=count)
                # model_day.save()

populate_model(extract(data,'day','pet'),'day','pet')

parrots Monday 1000
goldfish Monday 1000
canary Monday 1000
turtles Monday 999
hamster Monday 999
rabbit Monday 1000
cats Monday 999
dogs Monday 1000


2.	Que mascota tiene el mayor número de comentarios positivos, negativos o neutros.

In [162]:
def populate_sentiment_model(arg_df, w_arg, f_arg):
    for pet in arg_df[f_arg].unique():
        df = arg_df[arg_df.pet == pet]
        activity = df.pivot_table(index=[w_arg], aggfunc='size')
        for sent in df[w_arg].unique():
            count = activity[sent]
                # model_sentiment = TempSentiment(pet=str(pet), sentiment=str(day), count=int(count))
                # model_sentiment.save()
populate_sentiment_model(extract(data,'sentiment','pet'),'sentiment','pet')

In [163]:
sentiment_info=[]
for t in data.values():
    sentiment_info.append([t['pet'], t['sentiment']])

df_sentiment=pd.DataFrame(sentiment_info)
df_sentiment

,0,1
0,parrots,1
1,parrots,-1
2,parrots,0
3,parrots,-1
4,parrots,1
...,...,...
8007,cats,0
8008,cats,1
8009,cats,1
8010,cats,1


3.	Los hashtags que más se usaron por mascota.

In [164]:
tag_info=[]
for t in data.values():
    tags=[i for i in t['clean_text'].split() if len(i)>3]
    tag_info.append([t['pet'],tags ])
df_tag=pd.DataFrame(tag_info, columns=['pet','clean_text'])
df_tag

,pet,clean_text
0,parrots,"[I've, just, learned, that, some, pirates, lit..."
1,parrots,"[Therese, Coffey, really, there, Olympic, meda..."
2,parrots,"[Just, learned, about, existence, DRACULA, PAR..."
3,parrots,"[Therese, Coffey, really, there, Olympic, meda..."
4,parrots,"[Ssunflower, seeds, popular, especially, parrots]"
...,...,...
8007,cats,"[Ñaña, salgas, “prófugos, sueltos, poner, cara..."
8008,cats,"[crocs, really, just, swamp, cats]"
8009,cats,"[Trashy, people, toss, away, very, senior, cat..."
8010,cats,"[crocs, really, just, swamp, cats]"


In [165]:
# # df_sentiment=extract(data,'clean_text','pet')
# # df_day=most_of(df_day,'day')
# # df_day=most_of(extract(data,'day'),'day')
# dfl=pd.Series(' '.join(df_tag['hashtags']).lower().split()).value_counts()[:10]
# debug(dfl)

In [167]:
full_list = []  # list containing all words of all texts
for elmnt in df_tag['clean_text']:  # loop over lists in df
    full_list += elmnt  # append elements of lists to full list
val_counts = pd.Series(full_list).value_counts()[:5]
debug(list(val_counts.index))

type:  <class 'list'>
len:  5
['just', 'Hamster', 'with', 'that', 'have']


In [179]:
tag_info=[]
for t in df_tag['pet'].unique():
    df = df_tag[df_tag.pet == t]
    debug(df)
    # tag_info.append([t['pet'],tags ])
# df_tag=pd.DataFrame(tag_info, columns=['pet','clean_text'])
# df_tag


type:  <class 'pandas.core.frame.DataFrame'>
len:  1000
          pet                                         clean_text
0     parrots  [I've, just, learned, that, some, pirates, lit...
1     parrots  [Therese, Coffey, really, there, Olympic, meda...
2     parrots  [Just, learned, about, existence, DRACULA, PAR...
3     parrots  [Therese, Coffey, really, there, Olympic, meda...
4     parrots  [Ssunflower, seeds, popular, especially, parrots]
...       ...                                                ...
7978  parrots  [Since, crowdfunding, #operationOBP, 2016, we'...
7981  parrots  [Therese, Coffey, really, there, Olympic, meda...
7982  parrots  [_ayy, Parrots, really, killing, goth, aesthetic]
7984  parrots  [Audubon, condemns, murder, Gonzalo, Cardona, ...
7986  parrots  [Funny, only, pipe, this, time, year, every, y...

[1000 rows x 2 columns]
type:  <class 'pandas.core.frame.DataFrame'>
len:  1000
           pet                                         clean_text
358   goldfish  [